In [ ]:
import csv
from tqdm import trange, tqdm_notebook
from pandas import ExcelWriter
import datetime, warnings  
import matplotlib as mpl

from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec

import numpy as np
import pandas as pd

import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [ ]:
weather = pd.read_csv('C:/Users/Playdata/py_projects/Mentoring/input/HourlyData2015.txt', low_memory=False)

In [ ]:
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 
    for x in unique_list: 
        return unique_list

In [ ]:
# df = pd.read_csv('C:/Users/Playdata/py_projects/Mentoring/input/flights.csv', low_memory=False)
df_new = pd.read_csv('C:/Users/Playdata/py_projects/Mentoring/input/df_new.csv', low_memory=False, sep ='\t')

In [ ]:
airport_data = pd.read_csv('C:/Users/Playdata/py_projects/Mentoring/SiteList.csv')

# for making df_new with df

In [ ]:
df_OA = unique(df_new['ORIGIN_AIRPORT'])

In [ ]:
len(df_OA) #628
print(type(df_OA))

In [ ]:
wd_OA = unique(weather['Site'])
len(wd_OA) # 405
print(wd_OA)

In [ ]:
in_both_4ch= []
in_both_dict = {}

In [ ]:
for code_4 in wd_OA:
    a = code_4[1:4]
    b = code_4[0:3]
    if  a in df_OA:
        in_both_4ch.append(code_4)
        key = code_4
        value = a
        in_both_dict[code_4] = a
    elif b in df_OA:
        in_both_4ch.append(code_4)
        key = code_4
        value = b
        in_both_dict[code_4] = b

In [ ]:
def condi(x):
    if x in in_both_4ch:
        return True
    else:
        return False

In [ ]:
weather = weather[weather['Site'].apply(condi)]

In [ ]:
weather[:5] #1787967 rows × 13 columns

In [13]:
def from4to3(x):
    if x in in_both_dict:
        return in_both_dict[x]
    else:
        return None

In [14]:
weather['Site_3ch'] = weather['Site'].apply(from4to3)

In [15]:
weather = weather[['Site_3ch', 'Date', 'Hour', 'Temperature', 'Dewpoint', 'RH', 'WindDir',
       'Windspeed', 'CldFrac', 'MSLP', 'Weather', 'Precip', 'Source']]

In [16]:
weather['Date'] =  pd.to_datetime(weather['Date'], infer_datetime_format =True)

In [17]:
hours = list(weather['Hour'].values)
dates = weather['Date']

# hours = dates.tolist()
# aaa
aa = []
for i in dates:
    a = '{:%Y-%m-%d}'.format(i)
    aa.append(a)
# aa

hrs=[]
for t in hours:
    t = str("{0:02d}".format(int(t))) + ':00:00'
    hrs.append(t)
    

In [18]:
a = pd.DataFrame(aa, columns=list('a'))
b = pd.DataFrame(hrs, columns=list('a'))
datetime = a+' '+b
datetime['a'] = pd.to_datetime(datetime['a'],infer_datetime_format=True)
weather['a'] = list(datetime['a'])
weather.rename(columns = {'a' : 'datetime'}, inplace = True)
weather['datetime'] =  pd.to_datetime(weather['datetime'], infer_datetime_format =True)


In [19]:
weather.drop(columns = 'Source', inplace = True)

In [20]:
def whether_delayed(time):
    if time >= 120:
        return 'y'
    else:
        return 'n'

df_new['DEPARTURE_DELAYED_YN'] = df_new['DEPARTURE_DELAY'].apply(whether_delayed)
df_new['SCHEDULED_DEPARTURE'] = pd.to_datetime(df_new['SCHEDULED_DEPARTURE'])

In [21]:
df_new['DATETIME'] = df_new['SCHEDULED_DEPARTURE'].dt.round("H")

In [22]:
weather = weather[['Site_3ch', 'datetime','Temperature','Dewpoint','RH','WindDir','Windspeed','CldFrac','MSLP','Precip']]

In [23]:
df_new.columns
weather.columns

Index(['Unnamed: 0', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'WEATHER_DELAY', 'CANCELLED',
       'CANCELLATION_REASON', 'DEPARTURE_DELAYED_YN', 'DATETIME'],
      dtype='object')

Index(['Site_3ch', 'datetime', 'Temperature', 'Dewpoint', 'RH', 'WindDir',
       'Windspeed', 'CldFrac', 'MSLP', 'Precip'],
      dtype='object')

In [24]:
df_new.isnull().sum()
weather.isnull().sum()

Unnamed: 0                    0
AIRLINE                       0
ORIGIN_AIRPORT                0
DESTINATION_AIRPORT           0
SCHEDULED_DEPARTURE           0
DEPARTURE_TIME            86153
DEPARTURE_DELAY           86153
SCHEDULED_TIME                6
ELAPSED_TIME             105071
WEATHER_DELAY           4755640
CANCELLED                     0
CANCELLATION_REASON     5729195
DEPARTURE_DELAYED_YN          0
DATETIME                      0
dtype: int64

Site_3ch           0
datetime           0
Temperature        0
Dewpoint           0
RH                 0
WindDir            0
Windspeed          0
CldFrac         8761
MSLP            8761
Precip         17854
dtype: int64

In [25]:
weather.rename(columns = {'datetime' : 'DATETIME'}, inplace = True)

In [ ]:
for site in unique(df_new['ORIGIN_AIRPORT']):
    print('----------------------------------------------------')
    print(str(site) + '공항 시작')
    if site in unique(weather['Site_3ch']):

        dfdf = df_new[df_new['ORIGIN_AIRPORT'] == site]
        wewe = weather[weather['Site_3ch'] == site]

        FIFI = pd.merge(dfdf, wewe, on='DATETIME')
        FIFI.rename(columns = {'datetime' : 'DATETIME',
                               'Temperature' : 'TEMPERATURE',
                                'WindDir' : 'WIND_DIRECTION',
                                'Windspeed' : 'WIND_SPEED',
                                'Dewpoint' : 'DEW_POINT',
                                'RH' : 'RELATIVE_HUMIDITY',
                                'CldFrac' : 'CLOUD_FRACTION',
                                'MSLP' : 'MEAN_SEA_LEVEL_PRESSURE',
                                'Precip' : 'PRECIPITATION'
                               }, inplace = True)

        if FIFI.shape[0] != 0:
            for i in trange(12): 
                i += 1
                try:
                    a = sns.pairplot(FIFI[FIFI['DATETIME'].dt.month == i][['TEMPERATURE', 'WIND_DIRECTION','WIND_SPEED','DEW_POINT','RELATIVE_HUMIDITY','CLOUD_FRACTION','MEAN_SEA_LEVEL_PRESSURE', 'PRECIPITATION', 'DEPARTURE_DELAYED_YN']], hue = 'DEPARTURE_DELAYED_YN', palette = { 'y':'r', 'n': 'b'})
                    a.savefig('pairplot_AIRPORT_' + str(site) + '_Month_' + str(i) + '_qunant_varialbe.png')
                except:
                    print(str(site) + '공항' + str(i) + '월 자료 없음')
        else:
            print('오류오류오륭ㄹ오')
    else:
        print( str(site) + ' = 해당 공항 날씨 데이터 없음')




In [26]:
FIFIFI = pd.DataFrame(columns = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'WEATHER_DELAY', 'CANCELLED',
       'CANCELLATION_REASON', 'DEPARTURE_DELAYED_YN', 'DATETIME', 'TEMPERATURE', 'WIND_DIRECTION','WIND_SPEED','DEW_POINT','RELATIVE_HUMIDITY','CLOUD_FRACTION','MEAN_SEA_LEVEL_PRESSURE', 'PRECIPITATION', 'LATITUDE','LONGTITUE','FEET', 'DEPARTURE_DELAYED_YN'])

In [42]:
weather.columns

Index(['Site_3ch', 'DATETIME', 'Temperature', 'Dewpoint', 'RH', 'WindDir',
       'Windspeed', 'CldFrac', 'MSLP', 'Precip'],
      dtype='object')

In [43]:
airport_data.columns

Index(['Site4', 'City', 'TimeZone', 'OffUTC', 'State', 'Lat', 'Lon',
       'Elevation(feet)', 'Site_3ch'],
      dtype='object')

In [41]:
df_new.columns

Index(['Unnamed: 0', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'WEATHER_DELAY', 'CANCELLED',
       'CANCELLATION_REASON', 'DEPARTURE_DELAYED_YN', 'DATETIME'],
      dtype='object')

In [32]:
airport_data[:5]

,Site4,City,TimeZone,OffUTC,State,Lat,Lon,Elevation(feet)
0,CWAJ,ERIEAU (MAPS),EST,-5,ON,42.250000,-81.900000,584
1,CWBE,KILLARNEY (MAPS),EST,-5,ON,45.966667,-81.466667,643
2,CWBZ,ST ANICET,EST,-5,QC,45.116667,-74.266667,161
3,CWFJ,CARDSTON (AUT),MST,-7,AB,49.200000,-113.266667,3727
4,CWGD,GODERICH (AUTO8),EST,-5,ON,43.766667,-81.716667,702


In [34]:
airport_data['Site_3ch'] = airport_data['Site4'].apply(from4to3)

In [ ]:
for site in unique(df_new['ORIGIN_AIRPORT']):
    print('----------------------------------------------------')
    print(str(site) + '공항 시작')
    if site in unique(weather['Site_3ch']):

        dfdf = df_new[df_new['ORIGIN_AIRPORT'] == site]
        wewe = weather[weather['Site_3ch'] == site]
        adad  = airport_data[airport_data['Site_3ch'] == site] 
        FIFI = pd.merge(dfdf, wewe, on='DATETIME')
        FIFIFI = pd.merge(FIFI, adad, on = 'Site_3ch')
        FIFI.rename(columns = {'datetime' : 'DATETIME',
                               'Temperature' : 'TEMPERATURE',
                                'WindDir' : 'WIND_DIRECTION',
                                'Windspeed' : 'WIND_SPEED',
                                'Dewpoint' : 'DEW_POINT',
                                'RH' : 'RELATIVE_HUMIDITY',
                                'CldFrac' : 'CLOUD_FRACTION',
                                'MSLP' : 'MEAN_SEA_LEVEL_PRESSURE',
                                'Precip' : 'PRECIPITATION',
                                'Lat' : 'LATITUDE',
                                'Lon' : 'LONGTITUDE',
                                'Elevation(feet)' : 'FEET'
                               }, inplace = True)
        if FIFIFI.shape[0] != 0:
            try:
                a = sns.pairplot(FIFIFI[FIFIFI['ORIGIN_AIRPORT'] == site][['TEMPERATURE', 'WIND_DIRECTION','WIND_SPEED','DEW_POINT','RELATIVE_HUMIDITY','CLOUD_FRACTION','MEAN_SEA_LEVEL_PRESSURE', 'PRECIPITATION', 'LATITUDE','LONGTITUDE','FEET', 'DEPARTURE_DELAYED_YN', 'LATITUDE','LONGTITUDE','FEET']], hue = 'DEPARTURE_DELAYED_YN', palette = { 'y':'r', 'n': 'b'})
                a.savefig('pairplot_AIRPORT_' + str(site) + '_qunant_varialbe_with_lat_lon.png')
            except:
                print(str(site) + '공항' + str(i) + '월 자료 없음')
            
#             ##per month
#             for i in trange(12): 
#                 i += 1
#                 try:
#                     a = sns.pairplot(FIFI[FIFI['DATETIME'].dt.month == i][['TEMPERATURE', 'WIND_DIRECTION','WIND_SPEED','DEW_POINT','RELATIVE_HUMIDITY','CLOUD_FRACTION','MEAN_SEA_LEVEL_PRESSURE', 'PRECIPITATION', 'LATITUDE','LONGTITUDE','FEET', 'DEPARTURE_DELAYED_YN', 'LATITUDE','LONGTITUDE','FEET']], hue = 'DEPARTURE_DELAYED_YN', palette = { 'y':'r', 'n': 'b'})
#                     a.savefig('pairplot_AIRPORT_' + str(site) + '_Month_' + str(i) + '_qunant_varialbe_with_lat_lon.png')
#                 except:
#                     print(str(site) + '공항' + str(i) + '월 자료 없음')
#         else:
#             print('오류오류오륭ㄹ오')
#     else:
#         print(str(site) + ' = 해당 공항 날씨 데이터 없음')